In [2]:
import pandas as pd, numpy as np 
pd.set_option('display.max_columns', None)
import sqlalchemy, sqlite3 

In [3]:
season = "2007_08" 
dir = f"../../static/CSV/{season}/" 

names = ['pergame_'+season, 'advanced_'+season, 'per100_'+season, 'playbyplay_'+season, 'totals_'+season]  
dfs = {} 

invalid = ['-9999', 'Unnamed', 'Rk', 'Player-additional']

# Yuck I know triple nested for loop 
for name in names:
    dfs[name] = pd.read_csv(dir+name+'.csv') 
    for inv in invalid:
        for col in dfs[name].columns:
            if inv in col or col in invalid:
                
                dfs[name] = dfs[name].drop([col], axis=1)
                
dfs['totals_'+season] = dfs['totals_'+season].loc[:,['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'MP', 'FG', 'FGA', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB']] 
dfs['totals_'+season].columns = 'T' + dfs['totals_'+season].columns 
dfs['totals_'+season]['Player'] =  dfs['pergame_'+season]['Player']
dfs['totals_'+season] 

,TPTS,TTRB,TAST,TSTL,TBLK,TTOV,TPF,TMP,TFG,TFGA,T3PA,T2P,T2PA,TFT,TFTA,TORB,TDRB,Player
0,10,10,4,1,0,1,9,51,3,14,0,3,14,4,4,6,4,Shareef Abdur-Rahim
1,276,137,52,31,8,34,83,970,99,241,48,89,193,68,87,36,101,Arron Afflalo
2,51,13,8,0,1,6,17,165,21,65,19,18,46,6,12,4,9,Maurice Ager
3,70,19,19,6,0,16,20,177,15,57,34,5,23,30,31,1,18,Blake Ahearn
4,1350,578,122,55,94,126,240,2649,561,1160,7,560,1153,227,298,220,358,LaMarcus Aldridge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,347,220,60,29,41,29,79,1104,143,293,11,139,282,57,69,48,172,Dorell Wright
447,223,117,40,29,12,34,43,640,90,169,24,80,145,33,52,33,84,Julian Wright
448,15,38,4,3,3,4,31,161,6,21,0,6,21,3,6,8,30,Lorenzen Wright
449,561,115,63,35,7,94,130,1158,210,478,100,170,378,101,124,23,92,Nick Young


In [7]:
def combine_dfs(dfs, names):
    curr = None
    for name in names:
        if curr is None: 
            curr = dfs[name]
        else:
            diff = dfs[name].columns.difference(curr.columns).tolist()
            diff.append('Player') 
            # print(diff)
            curr = pd.merge(curr, dfs[name].loc[:,diff], on = 'Player') 
    return curr     

In [8]:
master = combine_dfs(dfs, names) 
master

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,DRtg,ORtg,And1,BadPass,Blkd,C%,LostBall,Off.,Off..1,On-Off,OnCourt,PF%,PG%,PGA,SF%,SG%,Shoot,Shoot.1,T2P,T2PA,T3PA,TAST,TBLK,TDRB,TFG,TFGA,TFT,TFTA,TMP,TORB,TPF,TPTS,TSTL,TTOV,TTRB
0,Shareef Abdur-Rahim,PF,31,SAC,6,0,8.5,0.5,2.3,0.214,0.0,0.0,NaN,0.5,2.3,0.214,0.214,0.7,0.7,1.000,1.0,0.7,1.7,0.7,0.2,0.0,0.2,1.5,1.7,0.000,11.1,0.0,-6.9,-2.5,9.1,0.0,0.286,-4.4,14.2,0.0,5.5,1.0,6.0,11.6,0.317,14.7,-0.1,0.0,-0.005,114,95,0,0,3,32%,0,0,1,-1.0,-3.3,69%,NaN,9,NaN,NaN,5,1,3,14,0,4,0,4,3,14,4,4,51,6,9,10,1,1,10
1,Arron Afflalo,SG,22,DET,75,9,12.9,1.3,3.2,0.411,0.1,0.6,0.208,1.2,2.6,0.461,0.432,0.9,1.2,0.782,0.5,1.3,1.8,0.7,0.4,0.1,0.5,1.1,3.7,0.199,8.1,0.7,-1.5,1.0,12.5,1.4,0.361,-2.6,4.4,0.5,10.2,1.8,10.9,8.4,0.494,15.2,0.1,1.9,0.092,104,104,7,10,31,NaN,8,6,2,0.2,8.7,NaN,NaN,113,5%,95%,32,33,89,193,48,52,8,101,99,241,68,87,970,36,83,276,31,34,137
2,Maurice Ager,SG,23,TOT,26,3,6.3,0.8,2.5,0.323,0.1,0.7,0.158,0.7,1.8,0.391,0.346,0.2,0.5,0.500,0.2,0.3,0.5,0.3,0.0,0.0,0.2,0.7,2.0,0.292,8.0,0.4,-11.1,-4.2,6.3,0.0,0.185,-6.8,2.8,-0.4,1.8,0.0,7.9,4.6,0.363,21.3,-0.4,-0.4,-0.114,114,77,2,2,7,NaN,1,3,3,-11.4,-11.8,10%,NaN,17,35%,55%,9,6,18,46,19,8,1,9,21,65,6,12,165,4,17,51,0,6,13
3,Blake Ahearn,PG,23,MIA,12,0,14.8,1.3,4.8,0.263,0.8,2.8,0.294,0.4,1.9,0.217,0.351,2.5,2.6,0.968,0.1,1.5,1.6,1.6,0.5,0.0,1.3,1.7,5.8,0.596,17.2,0.0,-4.8,-1.3,12.4,0.1,0.544,-3.5,0.7,-0.1,9.0,1.8,18.5,6.4,0.495,23.2,-0.1,0.0,-0.006,111,94,2,11,3,NaN,5,0,2,-15.1,-23.9,NaN,57%,43,NaN,43%,9,8,5,23,34,19,0,18,15,57,30,31,177,1,20,70,6,16,19
4,LaMarcus Aldridge,PF,22,POR,76,76,34.9,7.4,15.3,0.484,0.0,0.1,0.143,7.4,15.2,0.486,0.484,3.0,3.9,0.762,2.9,4.7,7.6,1.6,0.7,1.2,1.7,3.2,17.8,0.006,8.8,2.8,0.2,-0.4,15.8,2.8,0.257,0.6,9.8,3.5,18.5,1.1,8.9,12.8,0.523,25.3,1.5,6.2,0.113,107,108,29,22,46,27%,46,37,13,2.9,-0.3,73%,NaN,294,NaN,NaN,123,144,560,1153,7,122,94,358,561,1160,227,298,2649,220,240,1350,55,126,578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,Dorell Wright,SF,22,MIA,44,34,25.1,3.3,6.7,0.488,0.1,0.3,0.364,3.2,6.4,0.493,0.495,1.3,1.6,0.826,1.1,3.9,5.0,1.4,0.7,0.9,0.7,1.8,7.9,0.038,9.4,3.0,0.1,0.7,19.0,1.1,0.235,-0.7,5.1,1.2,14.4,1.4,8.2,11.9,0.537,15.1,0.6,2.2,0.096,108,111,6,18,11,NaN,3,3,5,4.2,-6.4,29%,NaN,131,71%,NaN,48,34,139,282,11,60,41,172,143,293,57,69,1104,48,79,347,29,29,220
477,Julian Wright,SF,20,NOH,57,1,11.2,1.6,3.0,0.533,0.2,0.4,0.417,1.4,2.5,0.552,0.562,0.6,0.9,0.635,0.6,1.5,2.1,0.7,0.5,0.2,0.6,0.8,3.9,0.142,9.5,1.5,1.8,1.5,15.7,0.9,0.308,0.3,5.9,0.7,15.4,2.4,15.1,10.7,0.581,16.4,0.6,1.7,0.126,104,111,4,20,8,NaN,10,2,2,-4.9,2.0,33%,NaN,89,62%,5%,30,22,80,145,24,40,12,84,90,169,33,52,640,33,43,223,29,34,117
478,Lorenzen Wright,C,32,TOT,18,1,8.9,0.3,1.2,0.286,0.0,0.0,NaN,0.3,1.2,0.286,0.286,0.2,0.3,0.500,0.4,1.7,2.1,0.2,0.2,0.2,0.2,1.7,0.8,0.000,3.5,1.4,-6.6,0.3,21.6,0.2,0.286,-6.9,5.8,-0.2,2.5,1.0,14.5,13.8,0.317,7.8,-0.2,0.0,0.001,108,79,0,0,1,94%,0,3,1,-2.9,-4.9,6%,NaN,10,NaN,NaN,17,3,6,21,0,4,3,30,6,21,3,6,161,8,31,15,3,4,38
479,Nick Young,SG,22,WAS,75,2,15.4,2.8,6.4,0.439,0.5,1.3,0.400,2.3,5.0,0.450,0.481,1.3,1.7,0.815,0.3,1.2,1.5,0.8,0.5,0.1,1.3,1.7,7.5,0.209,9.6,0.5,-4.7,-2.1,9.6,0.6,0.259,-2.6,2.3,-0.1,11.6,1.6,15.0,5.8,0.527,24.9,-0.8,0.4,0.017,112,98,10,22,26,NaN,33,13,4,-11.6,-8.6,NaN,NaN,142,14%,86%,55,58,170,378,100,63,7,92,210,478,101,124,1158,23,130,561,35,94,115


In [9]:
engine = sqlalchemy.create_engine('sqlite:///../../DB/ballbase.db') 
for name in names:
    dfs[name].to_sql(name, con = engine, if_exists = 'replace', index = False)  

In [11]:
master.to_sql('master_'+season, con=engine, if_exists='replace', index=False) 
'master_'+season

'master_2007_08'